#### José Manuel Ramírez Araiza. 02/09/23

# Tarea - Programación lineal

### Diseño de la Dieta Óptima

Se quiere producir comida para gatos de la manera más barata, no obstante se debe también asegurar que se cumplan los datos requeridos de analisis nutricional. Por lo que se quiere variar la cantidad de cada ingrediente para cumplir con los estandares nutricionales. Los requisitos que se tienen es que en $100$ gramos, se deben tener **por lo menos** $8$ gramos de proteína y $6$ gramos de grasa. Así mismo, no se debe tener más de $2$ gramos de fibra y $0.4$ gramos de sal.  

Los datos nutricionales se pueden obtener de la siguiente tabla:

Ingrediente|Proteína|Grasa|Fibra|Sal
:----|----|----|----|----
Pollo|  10.0%|08.0%|00.1%|00.2%
Carne|  20.0%|10.0%|00.5%|00.5%
Cordero|15.0%|11.0%|00.5%|00.7%
Arroz|  00.0%|01.0%|10.0%|00.2%
Trigo|  04.0%|01.0%|15.0%|00.8%
Gel|    00.0%|00.0%|00.0%|00.0%

Los costos de cada producto son:

Ingrediente|Costo por gramo
:----|----
Pollo|$\$$0.013
Carne|$\$$0.008
Cordero|$\$$0.010
Arroz|$\$$0.002
Trigo|$\$$0.005
Gel|$\$$0.001    

Lo que se busca optimizar en este caso es la cantidad de productos que se debe utilizar en la comida de gato, minimizando el costo total. Para simplificar la notación use las siguientes variables: 

+ $x_1:$ Gramos de pollo  
+ $x_2:$ Gramos de carne  
+ $x_3:$ Gramos de cordero  
+ $x_4:$ Gramos de arroz  
+ $x_5:$ Gramos de trigo  
+ $x_6:$ Gramos de gel

La tarea consiste en plantear el problemade programación lineal que permita satisfacer las necesidades alimenticias del gato al tiempo que minimice el costo total y resolverlo con `linprog`.

### Solución
1. Minimizar
$$ Z=0.013x_1+0.008x_2+0.010x_3+0.002x_4+0.005x_5+0.001x_6 $$
2. Restricciones:

- a) De proteína:
$$ 10x_1+2x_2+15x_3+0x_4+4x_5+0x_6>=8 $$
- b) De grasa:
$$ 8x_1+10x_2+11x_3+1x_4+1x_5+0x_6>=6 $$
- c) De fibra: 
$$ 0.1x_1+0.5x_2+0.5x_3+10x_4+15x_5+0x_6<=2 $$
- d) De sal:
$$ 0.2x_1+0.5x_2+0.7x_3+0.2x_4+0.8x_5+0x_6<=0.4 $$
- e) De cantidades no negativas:
$$ x_1,x_2,x_3,x_4,x_5,x_6>=0 $$

In [4]:
import numpy as np
import scipy.optimize as opt

# Coeficientes de la función objetivo (costo por gramo)
c = np.array([0.013, 0.008, 0.010, 0.002, 0.005, 0.001])

# Coeficientes de las restricciones de desigualdad (nutrientes en 100 gramos)
# Los coeficientes son negativos porque estamos llevando todo al lado izquierdo de las desigualdades.
A = np.array([
    [-0.10, -0.20, -0.15, 0.00, -0.04, 0.00],  # Proteína
    [-0.08, -0.10, -0.11, -0.01, -0.01, 0.00],  # Grasa
    [0.001, 0.005, 0.005, -0.10, -0.15, 0.00],  # Fibra
    [0.002, 0.005, 0.007, 0.002, 0.008, 0.00],  # Sal
    [1.00, 1.00, 1.00, 1.00, 1.00, 1.00],  # Total gramos en la lata
])

# Lados derecho de las restricciones de desigualdad (nutrientes requeridos)
b = np.array([-8, -6, 2, 0.4, 100])  # Agregamos la restricción de 100 gramos en la lata

# Límites de las variables (De cantidades no negativas)
x_bounds = [(0, None)] * 6  # x_i >= 0 para todas las variables

# Resolver el problema
res = opt.linprog(c, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')

if res.success:
    print("Solución óptima encontrada:")
    print("Gramos de pollo:", res.x[0])
    print("Gramos de carne:", res.x[1])
    print("Gramos de cordero:", res.x[2])
    print("Gramos de arroz:", res.x[3])
    print("Gramos de trigo:", res.x[4])
    print("Gramos de gel:", res.x[5])
    print("Costo total mínimo:", res.fun)
else:
    print("No se encontró una solución óptima.")



Solución óptima encontrada:
Gramos de pollo: 0.0
Gramos de carne: 60.0
Gramos de cordero: 0.0
Gramos de arroz: 0.0
Gramos de trigo: 0.0
Gramos de gel: 0.0
Costo total mínimo: 0.48


### Problema de distribución de energía eléctrica

La Comisión Federal de Electricidad **(CFE)** dispone de cuatro plantas de generación para satisfacer la demanda diaria eléctrica en cuatro ciudades, Guadalajara, León y Morelia. Las plantas $1$, $2$ y $3$ pueden satisfacer $80$, $40$ y $60$ millones de KW al día respectivamente. Las necesidades de las ciudades de Guadalajara, León y Morelia son de $70$, $40$ y $70$ millones de Kw al día respectivamente. 


Los costos asociados al envío de suministro energético por cada millón de Kw entre cada planta y cada ciudad son los registrados en la siguiente tabla. 

-|Guadalajara|León|Morelia
:----|----|----|----
Planta 1|5|2|7
Planta 2|3|6|6
Planta 3|6|1|2

Y por último, las restricciones del problema, van a estar dadas por las capacidades de oferta y demanda de cada planta (en millones de KW) y cada ciudad.

Para simplificar la notación use las siguientes variables:

+ $x_1$: Kw (en millones) distribuidos de la Planta 1 a Guadalajara
+ $x_2$: Kw (en millones) distribuidos de la Planta 1 a León
+ $x_3$: Kw (en millones) distribuidos de la Planta 1 a Morelia
+ $x_4$: Kw (en millones) distribuidos de la Planta 2 a Guadalajara
+ $x_5$: Kw (en millones) distribuidos de la Planta 2 a León
+ $x_6$: Kw (en millones) distribuidos de la Planta 2 a Morelia
+ $x_7$: Kw (en millones) distribuidos de la Planta 3 a Guadalajara
+ $x_8$: Kw (en millones) distribuidos de la Planta 3 a León
+ $x_9$: Kw (en millones) distribuidos de la Planta 3 a Morelia

La tarea consiste en plantear el problema de programación lineal que permita satisfacer las necesidades de todas las ciudades al tiempo que minimice los costos asociados a la distribución y resolverlo con `linprog`.

### Solución
1. Miinimizar
$$ Z=5x_1+2x_2+7x_3+3_x4+6x_5+6x_6+6x_7+x_8+2x_9 $$
2. Restricciones:
- a) De oferta de plantas:
$$ x_1+x_2+x_3<=80 (Planta 1) $$
$$ x_4+x_5+x_6<=40 (Planta 2) $$
$$ x_7+x_8+x_9<=60 (Planta 3) $$
- b) De demanda de ciudades:
$$ x_1+x_4+x_7>=70 (Guadalajara) $$
$$ x_2+x_5+x_8>=40 (León) $$
$$ x_3+x_6+x_9>=70 (Morelia) $$
- c) No puede haber envío negativo:
$$ x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9>=0 $$

In [11]:
import numpy as np
import scipy.optimize as opt

# Coeficientes de la función objetivo (costos de distribución)
c = np.array([5, 2, 7, 3, 6, 6, 6, 1, 2])

# Coeficientes de las restricciones de oferta de plantas
A_eq = np.array([
    [1, 1, 1, 0, 0, 0, 0, 0, 0],  # Planta 1
    [0, 0, 0, 1, 1, 1, 0, 0, 0],  # Planta 2
    [0, 0, 0, 0, 0, 0, 1, 1, 1],  # Planta 3
])

# Lados derecho de las restricciones de oferta de plantas
b_eq = np.array([80, 40, 60])

# Coeficientes de las restricciones de demanda de ciudades
A_ub = np.array([
    [-1, 0, 0, -1, 0, 0, -1, 0, 0],  # Guadalajara
    [0, -1, 0, 0, -1, 0, 0, -1, 0],  # León
    [0, 0, -1, 0, 0, -1, 0, 0, -1],  # Morelia
])

# Lados derecho de las restricciones de demanda de ciudades
b_ub = np.array([-70, -40, -70])

# Límites de las variables (No puede haber envío negativo)
x_bounds = [(0, None)] * 9  # x_i >= 0 para todas las variables


# Resolver
res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=x_bounds, method='highs')

if res.success:
    print("Solución óptima encontrada:")
    print("Kw distribuidos de la Planta 1 a Guadalajara:", res.x[0])
    print("Kw distribuidos de la Planta 1 a León:", res.x[1])
    print("Kw distribuidos de la Planta 1 a Morelia:", res.x[2])
    print("Kw distribuidos de la Planta 2 a Guadalajara:", res.x[3])
    print("Kw distribuidos de la Planta 2 a León:", res.x[4])
    print("Kw distribuidos de la Planta 2 a Morelia:", res.x[5])
    print("Kw distribuidos de la Planta 3 a Guadalajara:", res.x[6])
    print("Kw distribuidos de la Planta 3 a León:", res.x[7])
    print("Kw distribuidos de la Planta 3 a Morelia:", res.x[8])
    print("Costo total mínimo de distribución:", res.fun)
else:
    print("No se encontró una solución óptima.")
opt.linprog(c, A_eq, b_eq, A_ub, b_ub)

Solución óptima encontrada:
Kw distribuidos de la Planta 1 a Guadalajara: 30.0
Kw distribuidos de la Planta 1 a León: 40.0
Kw distribuidos de la Planta 1 a Morelia: 10.0
Kw distribuidos de la Planta 2 a Guadalajara: 40.0
Kw distribuidos de la Planta 2 a León: 0.0
Kw distribuidos de la Planta 2 a Morelia: 0.0
Kw distribuidos de la Planta 3 a Guadalajara: 0.0
Kw distribuidos de la Planta 3 a León: 0.0
Kw distribuidos de la Planta 3 a Morelia: 60.0
Costo total mínimo de distribución: 540.0


        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 540.0
              x: [ 3.000e+01  4.000e+01  1.000e+01  4.000e+01  0.000e+00
                   0.000e+00  0.000e+00  0.000e+00  6.000e+01]
            nit: 6
          lower:  residual: [ 3.000e+01  4.000e+01  1.000e+01  4.000e+01
                              0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              6.000e+01]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              6.000e+00  1.000e+00  6.000e+00  4.000e+00
                              0.000e+00]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf        inf        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00  0.000e+00  0.0